In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import re

In [2]:
output_folder_path = "../outputs/RealWorld/facebookMedium/bots/03"
input_folder_path="../../output/RealWorld/facebookMedium/bots/03"
#datasets=['astroPh', 'emailEU', 'facebook', 'facebookMedium', 'facebookArtist', 'gitInteractions', 'hepPh', 'lastfm', 'redditHyperlink', 'twitchDE', 'twitchENGB', 'twitchES', 'twitchFR', 'twitchPTBR', 'twitchRU']
#botRatio=['m0_', 'm25_', 'm50_', 'm75_', 'm100_', 'm125_', 'm150_', 'm175_', 'm200_']
datasets=['facebookMedium']
startRatio=['01', '02', '03', '04', '05']
rewireRatio=[ 'r10_', 'r20_', 'r30_', 'r40_', 'r50_', 'r60_', 'r70_', 'r80_', 'r90_']

In [3]:
def create_graphs(path, sRatio):
    files=os.listdir(path)
    data={}
    data2={}
    average={}

    botRatio=[]
    for file in files:
        if file.endswith(".txt"):
            match = re.search(r"_m(\d+)_", file)
        if match:
            result = match.group(0)
        if not result in botRatio:
            botRatio.append(result)
    botRatio=sorted(botRatio, key=lambda x: int(x.strip('_m')))
    print(botRatio)

    for rr in rewireRatio:
        data[rr]={}
        data2[rr]={}
        average[rr]={}
        for mat in botRatio:
            data[rr][mat]=[]
            data2[rr][mat]=[]
            for file in files:
                if mat in file and rr in file:
                    #print(path+" "+file)
                    with open(os.path.join(path, file), 'r') as f:
                        content = pd.read_csv(path+"/"+file, names=['Epoch', 'Pop', 'Frac', 'DiscEdge'], skiprows=1, sep=" ")

                        content=content.drop(['Epoch', 'Pop'], axis=1)    
                        finalPerc=content.iloc[-1, 1]
                        finalEpoch=content.iloc[-1, 0]
                        # if finalPerc>0.5:
                        #     content['Frac']=1-content['Frac']

                        data[rr][mat].append(content)
                        data2[rr][mat].append(finalEpoch)
                        
        fig=plt.figure(figsize=(15,10))
        ax=fig.add_subplot(111)

        # rs=[ 'r10', 'r20', 'r30', 'r40', 'r50', 'r60', 'r70', 'r80', 'r90']
        numberOfOutputs={item: 0 for item in botRatio}

        start_rgb = (255, 0, 0)  # Red
        mid_rgb = (0, 255, 0)    # Green
        end_rgb = (0, 0, 255)    # Blue
        num_elements=len(botRatio)
        mid_point = num_elements // 2

        gradient_red_to_green = [
            tuple(
                float((start_rgb[i] + (mid_rgb[i] - start_rgb[i]) * t / (mid_point - 1)) / 255)
                for i in range(3)
            )
            for t in range(mid_point)
        ]

        # Generate the gradient from green to blue
        gradient_green_to_blue = [
            tuple(
                float((mid_rgb[i] + (end_rgb[i] - mid_rgb[i]) * t / (num_elements - mid_point - 1)) / 255)
                for i in range(3)
            )
            for t in range(num_elements - mid_point)
        ]

        # Combine the two gradients
        gradient = gradient_red_to_green + gradient_green_to_blue

        # Map each item to an RGB value
        gradient_mapping = {item: rgb for item, rgb in zip(botRatio, gradient)}
        label_mapping = {item: f"{int(item.strip('_m')) / 10}%" for item in botRatio}
        print(gradient_mapping)
        print(label_mapping)

        
        for k in botRatio:
            v=data[rr][k]
            for serie in v:
                #print(serie)
                if numberOfOutputs[k]==0:
                    ax.plot(serie['Frac'].tolist(), serie['DiscEdge'].tolist(), color=gradient_mapping[k], label=label_mapping[k])
                else:
                    ax.plot(serie['Frac'].tolist(), serie['DiscEdge'].tolist(), color=gradient_mapping[k])
                numberOfOutputs[k]=numberOfOutputs.get(k)+1

        plt.title('Minority Opinion Density over Discordant Edge Count on facebookMedium with '+rr+' percent on Dynamic Network', fontsize='large')
        plt.xlabel("Minority Opinion Density    ($\\rho$)", fontsize='large')
        plt.ylabel("$l_{01}$", fontsize='large')
        plt.legend(loc='upper left', title="BotRatio ($\\alpha$)")
        plt.show()
        path_str=str(rr)
        figpath=output_folder_path+"/DiscEdgevsOpinion_"+path_str+".png"
        fig.savefig(figpath, dpi=300)

In [1]:
for dataset in datasets:
    # for sRatio in startRatio:
    #     path=output_folder_path+dataset+"/"+sRatio
    #     #print(path)
    #     create_graphs(input_folder_path, sRatio)
    create_graphs(input_folder_path, "40")

NameError: name 'datasets' is not defined